In [ ]:
import pandas as pd
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import pyregion
import pyarrow.parquet as pq


In [ ]:
# modify to look for stims instead of stars 

def first_and_last_indices_above_one(numbers):
    above_one_indices = [i for i, num in enumerate(numbers) if num > 0]
    return above_one_indices[0] if above_one_indices else None, above_one_indices[-1] if above_one_indices else None

def gaussian(x, A, mu, sigma):
    return A * np.exp(-(x - mu)**2 / (2 * sigma**2))

def perp_line(m, x1, y1, x): 
    y = m*(x-x1) + y1 
    return y 

def per_star(r, image, num): 
    
    width = 16
    
    y = round(r[num].coord_list[0])
    x = round(r[num].coord_list[1])

    cutout = image[0].data[x-width:x+width,y-width:y+width]
    rows, cols = np.where(cutout>1)
    fit = np.polyfit(rows, cols, 2)
    f = np.poly1d(fit)

    starlist = []
    
    x_vals = range(round(cutout.shape[0]/2-5),round(cutout.shape[0]/2+5))
    
    for x_val in x_vals: 

        deriv = f.deriv()
        slope_of_normal = -1/deriv(x_val)
        y1 = f(x_val)
        x1= x_val
        xr = range(x_val-8,x_val+12)

        y = [round(perp_line(slope_of_normal, x1, y1, x)) for x in xr]
        y = [31 if a >= 32 else 0 if a <= 0 else a for a in y]
        #plt.scatter(xr, y, marker ='x')
        coords = list(zip(xr,y))
        crosscut = [cutout[c[0],c[1]] for c in coords]
        
        try:
            popt, _ = curve_fit(gaussian, xr, crosscut, p0=[1, 0, 1])

            # Calculate the FWHM from the fitted parameters
            fwhm = 2 * np.sqrt(2 * np.log(2)) * popt[2]
            
            first_index, last_index = first_and_last_indices_above_one(crosscut)
            width = last_index-first_index 
            
            maxy = max(crosscut)
            rsum = sum(crosscut)-maxy
            rstd = np.std(crosscut)
            rmean = np.mean(crosscut)
            maxy2 = sorted(crosscut)[-2]
            numg0 = sum(i > 1 for i in crosscut)            
            numge1 = sum(i >= 1 for i in crosscut)


        except: 
            fwhm = 999
            
        row = {'star': num, 
               'x_val': x_val, 
               'fwhm': fwhm, 
               'maxy': maxy,
               'maxy2': maxy2,
               'crosscut': crosscut, 
               'rsum': rsum,
              'rmean': rmean,
              'rstd': rstd,
              'numg0': numg0,
              'numge1': numge1,
              'width': width,
              'first_index': first_index,
              'last_index': last_index} 
        
        starlist.append(row.copy())

        
    star_data = pd.DataFrame(starlist)
    
    return star_data 
        
def per_eclipse(eclipse, time, timer): 
    
    image = fits.open(f"/media/bekah/BekahA/backplane_widths/e{eclipse}/e{eclipse}-nd-b00-f0025-t00{time}0-g_dose.fits.gz")
    region_name = f"/media/bekah/BekahA/backplane_widths/e{eclipse}/regions_n_t{timer}.reg"
    r = pyregion.open(region_name)
    
    combined_results = []

    for star in range(len(r)): 
        try: 
            star_data = per_star(r, image, star)
            combined_results.append(star_data)
        except: 
            print("Poor fit.")
        
    eclipse_results = pd.concat(combined_results, axis=0)
    eclipse_results['eclipse'] = eclipse
    eclipse_results['time'] = timer

    return eclipse_results 

def all_data(dlist): 
    
    combined_results = []
    
    for d in dlist: 
        df = per_eclipse(d[0], d[1], d[2])
        combined_results.append(df)
        
                    
    return combined_results 


In [ ]:
fuvstim = pd.read_csv("/home/bekah/glcat/fuv_offset/FUVStim.csv")

In [ ]:
fuvstim

In [ ]:
plt.plot(fuvstim['Unnamed: 0'], fuvstim['globalcount'])

In [ ]:
plt.scatter(fuvstim['exptime'], fuvstim['stimcount'], s=.5)
plt.xlabel('fuv exptime')
plt.ylabel('stim count')

In [ ]:
plt.scatter(fuvstim['deadtime'], fuvstim['stimcount'], s=.5)
plt.xlabel('fuv deadtime')
plt.ylabel('stim count')

# flags 
http://www.galex.caltech.edu/wiki/Public:Documentation/Appendix_A
See above for detailed description.
        Unsigned 2-byte integer.
        Least significant bit = 0:
        0  - N/A (don't assume it's zero!)
        1  - N/A (don't assume it's zero!)
        2  - Skip
        3  - 0
        4  - 0
        5  - Stim
        6  - Masked
        7  - BadAsp
        8  - Range
        9  - Badwalk
        10 - N/A (don't assume it's zero!)
        11 - N/A (don't assume it's zero!)
        12 - N/A (don't assume it's zero!)
        13 - N/A (don't assume it's zero!)
        14 - N/A (don't assume it's zero!)
        15 - N/A (don't assume it's zero!)

In [ ]:
# stim events should be in extended photonlist files (those used for dose maps)
# http://www.galex.caltech.edu/wiki/Public:Documentation/Appendix_A

photonlist = pq.read_table("/media/bekah/BekahA/backplane_widths/e23346/e23346-nd-b00.parquet").to_pandas()


In [ ]:
stims = photonlist[photonlist["col"] <= 200]

In [ ]:
photonlist

In [ ]:
stims

In [ ]:
plt.hist2d(stims['row']*4,stims['col']*4, bins=200)

In [ ]:
def avg_stimpos(band, eclipse):
    """
    Define the mean detector stim positions.

    :param band: The band to return the average stim positions for,
        either 'FUV' or 'NUV'.

    :type band: str

    :param eclipse: The eclipse number to return the average stim positions for.

    :type eclipse: int

    :returns: dict -- A dict containing the average x and y positions of the
        four stims (for the requested band).
    """

    if band == "FUV":
        avgstim = {
            "x1": -2541.88,
            "x2": 2632.06,
            "x3": -2541.53,
            "x4": 2631.68,
            "y1": 2455.28,
            "y2": 2455.02,
            "y3": -2550.89,
            "y4": -2550.92,
        }

    elif band == "NUV":
        if eclipse >= 38268:
            # The average stim positions after the clock change (post-CSP).
            avgstim = {
                "x1": -2722.53,
                "x2": 2470.29,
                "x3": -2721.98,
                "x4": 2471.09,
                "y1": 2549.96,
                "y2": 2550.10,
                "y3": -2538.57,
                "y4": -2538.62,
            }
        else:
            # The average stim positions for pre-CSP data (eclipse 37423).
            avgstim = {
                "x1": -2722.27,
                "x2": 2468.84,
                "x3": -2721.87,
                "x4": 2469.85,
                "y1": 2453.89,
                "y2": 2453.78,
                "y3": -2565.81,
                "y4": -2567.83,
            }

    else:
        raise ValueError("No valid band specified.")

    return avgstim


# ------------------------------------------------------------------------------

# ------------------------------------------------------------------------------
def find_stims_index(x, y, band, eclipse, margin=90.001):
    """
    Given a list of detector x,y positions of events, returns four
        arrays that contain the indices of likely stim events for that stim,
        i.e., the first array contains positions for stim1, the second array has
        positions of stim2, etc.

        Example of how the return indexes are used: x[index1], y[index1] would
        give all of the event positions for stim1.

    :param x: Detector 'x' positions to identify likely stim events from.

    :type x: list

    :param y: Detector 'y' positions to identify likely stim events from.

    :type y: list

    :param band: The band to return the constants for, either 'FUV' or 'NUV'.

    :type band: str

    :param eclipse: The eclipse number to return the average stim positions for.

    :type eclipse: int

    :param margin: +/- extent in arcseconds defining search box

    :type margin: float

    :returns: tuple -- A four-element tuple containing arrays of indexes that
        correpond to the event positions for stim1, stim2, stim3, and stim4.
    """

    # [Future] This method could be programmed better. Consider using numpy
    # "where" and the logical '&' operator, instead of .nonzero()?

    # Plate scale (in arcsec/mm).
    pltscl = 68.754932
    # Plate scale in arcsec per micron.
    ASPUM = pltscl / 1000.0
    # [Future] Could define the plate scale (in both units) as a constant for
    # the module, since it is used in many places?

    x_as = np.array(x) * ASPUM
    y_as = np.array(y) * ASPUM

    avg = avg_stimpos(band, eclipse)

    index1 = (
        (x_as > (avg["x1"] - margin))
        & (x_as < (avg["x1"] + margin))
        & (y_as > (avg["y1"] - margin))
        & (y_as < (avg["y1"] + margin))
    ).nonzero()[0]
    index2 = (
        (x_as > (avg["x2"] - margin))
        & (x_as < (avg["x2"] + margin))
        & (y_as > (avg["y2"] - margin))
        & (y_as < (avg["y2"] + margin))
    ).nonzero()[0]
    index3 = (
        (x_as > (avg["x3"] - margin))
        & (x_as < (avg["x3"] + margin))
        & (y_as > (avg["y3"] - margin))
        & (y_as < (avg["y3"] + margin))
    ).nonzero()[0]
    index4 = (
        (x_as > (avg["x4"] - margin))
        & (x_as < (avg["x4"] + margin))
        & (y_as > (avg["y4"] - margin))
        & (y_as < (avg["y4"] + margin))
    ).nonzero()[0]

    return index1, index2, index3, index4



In [ ]:
y = photonlist['y']
x = photonlist['x']
pltscl = 68.754932
# Plate scale in arcsec per micron.
ASPUM = pltscl / 1000.0
# [Future] Could define the plate scale (in both units) as a constant for
# the module, since it is used in many places?

x_as = np.array(x) * ASPUM
y_as = np.array(y) * ASPUM

In [ ]:
min(x_as)

In [ ]:
max(x_as)

In [ ]:
plt.hist2d(y_as,x_as,bins=500)

In [ ]:
y = photonlist['y']
x = photonlist['x']
t = photonlist['t']
band = "NUV"
eclipse = 23346

i1, i2, i3, i4 = find_stims_index(x, y, band, eclipse, margin=90.001)

In [ ]:
i1

In [ ]:
i2

In [ ]:
xi1 = x[i1] 
yi1 = y[i1] 
t1 = t[i1]
xi2 = x[i2] 
yi2 = y[i2] 
t2 = t[i2]
xi3 = x[i3] 
yi3 = y[i3] 
t3 = t[i3]
xi4 = x[i4]
yi4 = y[i4] 
t4 = t[i4]

In [ ]:
plt.scatter(xi1,yi1)
plt.scatter(xi2,yi2)
plt.scatter(xi3,yi3)
plt.scatter(xi4,yi4)

In [ ]:
plt.scatter(xi4,yi4, s=.2, c=t4)
plt.colorbar()

In [ ]:
plt.scatter(xi3,yi3, s=.2, c=t3)
plt.colorbar()